In [37]:
#Verificação do exame pelo dicom

 
def validacao_exame(path_exame):
    ds_x = pydicom.read_file(path_exame)
    shape = ds_x.pixel_array.shape
    
    #shape tem que ser em um formato de tupla - representa projeção x largura x altura
    #shape tem que ter tamanho 3
    #largura e altura devem ser de 64
    if type(shape)==type((1,1,1)) and len(shape)==3 and shape[1:] == (64,64):
        print("exame ok")
    else:
        print("exame rejeitado")

    

### Classes


In [38]:
import numpy as np
class MinMaxNorm:
    """
    MinMaxNorm - Classe responsável pela Normalizção min max das imagens.
        
    """
    
    def __init__(self):
        self.max = 0

    def fit_transform(self,images):
        """
        Busca o valor maximo dos dados, salva o valor e retorna o conjunto de imagens normalizado entre 0 e 1.
            Args:
                Image:Matrix de images - Lista no formato de matriz que representa as imagens que vão ser normalizada.
                
            Return: 
                Imagens Normalizdas entre 0 e 1.
        """
        
        self.max = np.asarray(images).max()
        return np.asarray(images)/self.max

    def transform(self,images):
        """
        Transforma o conjunto de imagens em valores entre 0 e 1.
            Args:
                Image:Matrix de images - Lista no formato de matriz que representa as imagens que vão ser normalizada.
                
            Return: 
                Imagens Normalizdas entre 0 e 1.
        """
    
        return np.asarray(images)/self.max
    
    def inverse_transform(self,images):
        """
        Transforma o conjunto de imagens em seus valores originais.
            Args:
                Image:Matrix de images - Lista no formato de matriz que representa as imagens.
                
            Return: 
                Imagens com seus valores originais.
        """
        
        return np.asarray(images)*self.max
    
    def save(self,name="./model/_scaler.txt"):
        f = open(name, "a")
        f.write(str(self.max))
        f.close()
    
    def load(self,name="./model/_scaler.txt"):
        f = open(name, "r")
        self.max = int(f.read())
        f.close()
        
        
        
    

In [39]:
import tensorflow as tf
import pydicom
import numpy as np

class  Modelo:
    

    def __init__(self,path_model_deno,path_model_sino, modelo_arqui,modelo_arqui_corre):
         
        """

            Args:
                path_model_deno:String - Caminnho para os pesos do modelo de remoção de ruído.
                path_model_sino:String - Caminnho para os pesos do modelo de correção de contraste.
                
                modelo_arqui:Json - Json com as informações para montagem da arquitetura de remoção de ruído.
                modelo_arqui_corre:Json - Json com as informações para montagem da arquitetura de correção de contraste.
                

            Return: 
        """
        
        
        json_file = open(modelo_arqui, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        
        #Carregamento do Modelo Denoising
        self.model_deno = tf.keras.models.model_from_json(loaded_model_json)
        self.model_deno.load_weights(path_model_deno)
        
        #Carregamento do Modelo Correção de Constraste
        
        
        
        json_file = open(modelo_arqui_corre, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        
        self.model_sino = tf.keras.models.model_from_json(loaded_model_json)
        self.model_sino.load_weights(path_model_sino)
    
    
    def change_uids(self,ds):
        
        """
            Recebe um exame no  formato dicom e altera o cabeçalho:SOP Instance UID,
            Study Instance UID, Series Instance UID  e Frame of Reference UID.

            Args:
                ds:pydicom.dataset.FileDataset - Arquivo dicom que vai ter a modificação

            Return: 
                ds:pydicom.dataset.FileDataset - Arquivo dicom modificado.
        """
        #Todo exame vai ter esse prefixo
        root = "1.5.428.6.6.695432.3.507."
        #(0008, 0018) SOP Instance UID  
        ds[('0008','0018')].value = pydicom.uid.generate_uid(root)
        #(0020, 000d) Study Instance UID
        #ds[('0020','000d')].value = pydicom.uid.generate_uid(root)
        #(0020, 000e) Series Instance UID 
        ds[('0020','000e')].value = pydicom.uid.generate_uid(root)
        #(0020, 0052) Frame of Reference UID
        #ds[('0020','0052')].value = pydicom.uid.generate_uid(root)
        
        
        #-----------NOVO------------
        
        #(0008, 103e) Series Description
        ds[('0008', '103e')].value = "Denoising "+ ds[('0008', '103e')].value
        
        #-----------NOVO END------------
        
        
        return ds

        
                
    def predicao(self,path_exame,scaler,path_save="tmp"):
        
        """
            Recebe um exame e a versão dele sem ruído.

            Args:
                path_exame:String - Caminnho onde está o exame que vai ter a remoção do ruído.
                scaler:MinMaxNorm - Normalizador de dados min max 0 e 1 para das redes de denoising e refino.
                path_save:String - Caminh onde será salvo o exame com remoção do ruído.

            Return: 
        """
        #Nome final do arquivo que vai ser salva
        """
            Para a API isso provavelmente deve ser alterado.
        """
        filename = path_exame.split("/")[-1]
        
        #tamanho da imagem
        IMG_WIDTH = 64
        IMG_HEIGHT =64
        MAX_PROJ = 64

        
        #Quantidade de imagens de entrada da rede
        PROJ_QUANT_IN = 1
        PROJ_QUANT_IN_CROP = 1
        #Quantidade de imagen de saida da Rede
        PROJ_QUANT_OUT = 1
        
                
        """
        ds_x.pixel_array - Contém  os valores dos pixels do exame
        Para Exame de SPCET o formato é Projeções x Altura da imagem x Largura da Imagem
        """
        #leitura do arquivo Dicom que vai passar pela remoção de ruído
        ds_x = pydicom.read_file(path_exame)
        #Alteração do cabeçalho
        ds_x = self.change_uids(ds_x)
        
        #-------Predição Denoising------------
        
        
        #lista onde ficara salva o exame sem ruído
        pred = []
        #Imagem apenas com a regição de interesse
        imgs_x = ds_x.pixel_array

        
        #Fazendo a remoção de ruído para cada projeção
        for i in range(len(imgs_x)):
            image = imgs_x[i].reshape(IMG_WIDTH,IMG_HEIGHT).copy()
            
        
            #-----------NOVO------------

            image[image>scaler.max] = scaler.max

            #-----------NOVO END------------
            
            image = scaler.transform(image)


            mix_image = np.zeros((1,IMG_WIDTH,IMG_HEIGHT,PROJ_QUANT_IN))
            for idx in range(PROJ_QUANT_IN):
                mix_image[0,:,:,idx] = np.asarray(image)

            sr_img = self.model_deno.predict(mix_image,verbose = 0)
            sr_img = scaler.inverse_transform(tf.squeeze(sr_img))
            sr_img[sr_img <0] = 0
            pred.append(sr_img)

            imgs_x[i] = pred[i]
        
        
    
        #-------Predição Refinamento dos projeções------------

        pred = []

        for i in range(len(imgs_x)):
            teste_image = imgs_x[i].reshape(1,IMG_WIDTH,IMG_HEIGHT,1).copy()
            teste_image[teste_image>scaler.max] = scaler.max
            teste_image2 = scaler.transform(teste_image)

            sr_img = self.model_sino.predict(teste_image2,verbose = 0)
            sr_img = scaler.inverse_transform(tf.squeeze(sr_img))
            #sr_img = rev_log(tf.squeeze(sr_img))
            sr_img[sr_img <0] = 0
            pred.append(sr_img)

            imgs_x[i] = pred[i]
        
        #-------Salvando Exame com a Remoção de Ruído------------
        
        #Salvando os novos valores de pixels do exame no dicom
        ds_x.PixelData = imgs_x

        
        #Salvando o novo dicom no caminho escolhido
        ds_x.save_as("./"+path_save+"/Corrigido_"+filename)  


    

### Main

In [40]:
if __name__ == '__main__':

    #-----PESOS------
    #Caminho para os pesos do modelo de remoção de ruído
    path_model_deno="./model/EXP02_02_16frame_Fine_Tuning_.h5"
    #Caminho para os pesos do modelo de correção de sinograma
    path_model_sino="./model/EXP02_02__Refino_01_Fine_Tuning_.h5" 
    
    
    #-----ARQUITETURAS-----
    #Json com as informações para monstagem da arquitetura.
    modelo_arqui = "./model/modelo.json"
    #Json Correção
    modelo_arqui_corre = "./model/model att corre.json"
    
    
    #-----NORMALIZADOR-----
    #Path para o normalizador da rede denosing
    norm_deno = "./config/_scaler.txt"
    
    
    #-----Normalizador do denoising----
    #Inicialização do Normalizador 0 e 1.
    scaler = MinMaxNorm()
    #Carregando a configuração para o normalizador
    scaler.load(norm_deno)
    
    #Carregamento dos modelos para remoção do ruído
    model = Modelo(path_model_deno,path_model_sino, modelo_arqui,modelo_arqui_corre)




    ### PARA CADA EXAME ESSE TRECHO TEM QUE SER EXECUTADO
    #Caminho do exames que vai passar pela remoção do ruído
    path_exame = "./Exame Teste/18.dcm"
    #Execução da remoção do ruído.
    model.predicao(path_exame,scaler,path_save="tmp")


    #Apaga o objeto que contem os modelos para liberar a memoria utilizada
    del model
    tf.keras.backend.clear_session()
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    
    tf.keras.backend.clear_session()
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    
    tf.keras.backend.clear_session()
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()
